# WALLABY internal data access science

A notebook pre-filled with cells and scripts for querying the WALLABY internal release data via the public TAP interface. The notebook has the following sections

1. Setup
2. Source finding detection catalog
3. Use kinematic models

---

# 1. Setup 

In [ ]:
import os
import tarfile
import requests
import getpass
import pyvo as vo
from pyvo.auth import authsession, securitymethods
from astropy.io.votable import from_table, parse_single_table

### Authenticate

<span style="font-weight: bold; color: #FF0000;">⚠ Update the cell below with your username and enter your password</span>

In [ ]:
# Enter WALLABY user username and password

username = 'wallaby_user'
password = getpass.getpass('Enter your password')

In [ ]:
# Connect with TAP service

URL = "https://wallaby.aussrc.org/tap"
auth = vo.auth.AuthSession()
auth.add_security_method_for_url(URL, vo.auth.securitymethods.BASIC)
auth.credentials.set_password(username, password)
tap = vo.dal.TAPService(URL, session=auth)

# 2. Source finding

First we need to identify which internal release we want to access. The WALLABY team uses tags to classify source finding detections as internally released. You can view all of the tags by running the cells below. Then, we set the `tag_name` variable two cells below. This will be used later in the notebook, so once you know what data you would like to access, update this value accordingly.

In [ ]:
# Get all tags

query = "SELECT * FROM wallaby.tag"
votable = tap.search(query)
table = votable.to_table()
table

As an example, let us retrieve all sources from phase 2 pilot observations released as part of the NGC 5044 DR1 release by supplying the `NGC 5044 DR1` tag to the `wallaby.get_catalog()` function:

<span style="font-weight: bold; color: #FF0000;">⚠ Update the `tag_name` value here</span>

In [ ]:
# SELECT TAG

tag_name = "NGC 5044 DR1"

In [ ]:
# Retrieve catalog as Astropy table

query = """SELECT d.*, ivo_string_agg(t.name || ': ' || t.description, '; ') AS tags, ivo_string_agg(c.comment, '; ') AS comments
        FROM wallaby.detection d
        LEFT JOIN wallaby.tag_detection td ON d.id = td.detection_id 
        LEFT JOIN wallaby.tag t ON t.id = td.tag_id
        LEFT JOIN wallaby.comment c ON d.id = c.detection_id
        WHERE t.name IN ('Internal Data Release', '$TAG_NAME')
        GROUP BY d.id"""
query = query.replace('$TAG_NAME', tag_name)
result = tap.search(query)
table = result.to_table()
table

The source catalog returned by the function should have been printed above (if not, check for error messages) and is stored in the variable `table`. We can now use basic indexing to access different catalog entries. For example, `table["f_sum"]` will return the entire column of integrated flux measurements, and we can use `table["f_sum"][0]` etc. to extract the individual fluxes for each source. Likewise, `table[0]` will extract the entire first row of the catalog, i.e. a list of all parameters of the first source.

## Calculate Physical Parameters

The next example demonstrates how to retrieve certain parameters from the catalog and use basic arithmetic to convert some of the raw measurements made by SoFiA into physically meaningful parameters such as redshift or HI mass. These can be directly appended to the catalog as additional columns using `table["parameter_name"] = <expression>`.

In [ ]:
import numpy as np
import scipy.constants as const
from astropy.cosmology import FlatLambdaCDM

# Set up cosmology
f_rest = 1.42040575e+9;  # HI rest frequency in Hz
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)

# Calculate redshift
table["redshift"] = f_rest / table["freq"] - 1.0

# Calculate luminosity distance in Mpc and HI mass in solar masses
table["dl"] = cosmo.luminosity_distance(table["redshift"]).value
table["log_mhi"] = np.log10(49.7 * table["dl"] * table["dl"] * table["f_sum"])

# Calculate source rest frame velocity width in km/s
table["dv"] = const.c * (1.0 + table["redshift"]) * table["w20"] / f_rest / 1000.0

# Show our new parameters
table["name", "id", "redshift", "dl", "log_mhi", "dv"].pprint(max_width=-1)

## Create a Plot

Once we’ve done our analysis, we can the create plots of any of the parameters in our table. In this example, let us plot the logarithmic HI mass against redshift and additionally colour the data points by source rest frame velocity width. If desired, the resulting plot can be exported as a PDF file and then downloaded to your local computer, e.g. to use in a presentation or publication.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams["font.size"] = 16

plt.scatter(table["redshift"], table["log_mhi"], s=16, c=table["dv"], cmap="jet")
plt.xlabel(r"$z$")
plt.ylabel(r"$\log_{10}(M_{\rm HI} / M_{\odot})$")
cbar = plt.colorbar()
cbar.set_label(r"$\Delta v \; (\mathrm{km \, s}^{-1})$")
plt.xlim(0.0, 0.1)
plt.ylim(7.0, 11.0)
plt.grid(True)

# Uncomment the following line to make a PDF copy in the notebook folder for download
#plt.savefig("my_plot.pdf", format="pdf", bbox_inches="tight", pad_inches=0.05)

plt.show()

## Filtering the catalog

Once we have the catalog loaded into an Astropy table object, we can easily make selections to suit our scientific needs. The following examples illustrate how the catalog can be filtered by certain criteria such as parameter ranges or the presence of comments and tags.

**Example: Filter sources by parameter range**

In [ ]:
# Select all sources within a certain RA and Dec range

mask = (table["ra"] > 202.0) & (table["ra"] < 203.0) & (table["dec"] > -22.5) & (table["dec"] < -21.5)
table[mask].pprint()

## Plot detection products

To view the products (moment maps, spectra, etc.) for a given detection you will need to authenticate with the AusSRC. For those who do not have an account with the AusSRC please first contact:

* Lachlan Campbell (lachlan.campbell@csiro.au)
* Hasan Rayan (hasan.rayan@csiro.au)

to have one set up for them. Once you have an account, you will need to log in to the portal at https://wallaby.aussrc.org to have your credentials synced. Then you will be able to run this section of the notebook.

In [ ]:
# useful function for downloading table products (requires authentication)

def download_products(row, products_filename, chunk_size=8192):
    """Download products for a row of the table (a detection entry)
    
    """
    name = row['source_name']
    access_url = row['access_url']
    votable = parse_single_table(access_url)
    product_table = votable.to_table()
    url = product_table[product_table['description'] == 'SoFiA-2 Detection Products'][0]['access_url']
    with requests.get(url, auth=(username, password), stream=True) as r:
        r.raise_for_status()
        with open(products_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=chunk_size):
                f.write(chunk)
    print(f'Downloaded completed for {name}')
    return

def download_table_products(table, directory, chunk_size=8192):
    """Download WALLABY products from ADQL queried table

    """
    if not os.path.exists(directory):
        os.mkdir(directory)
    print(f'Saving products to {directory}')
    for row in table:
        name = row['source_name']
        products_filename = os.path.join(directory, f'{name}.tar')
        download_products(row, products_filename, chunk_size)
    print('Downloads complete')
    return

In [ ]:
# Select detection of interest and download product files

detection_row = table[0]
name = detection_row['source_name']
products_filename = f'{name}.tar'
download_products(detection_row, products_filename)

In [ ]:
# Check filesystem for file

assert os.path.exists(products_filename), 'Download error'
tf = tarfile.open(products_filename)
tf.extractall(name)
tf.close()

In [ ]:
!ls '{name}'

### Create plots

Once we have the product files for the detection of interest, we can plot them using familiar Python libraries. In the example below, we will plot the moment 0 map of a detection of interest.

In [ ]:
import glob 
from astropy.wcs import WCS
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# plot summary figure

files = glob.glob(os.path.join(name, '*plot.png'))
file = files[0]
img = mpimg.imread(file)

plt.imshow(img) 
plt.axis('off')
plt.show()

In [ ]:
# open moment 0 map and plot

files = glob.glob(os.path.join(name, '*mom0.fits'))
file = files[0]
hdu = fits.open(file)[0]
wcs = WCS(hdu.header)

plt.subplot(projection=wcs) 
plt.imshow(hdu.data, origin='lower') 
plt.grid(color='white', ls='solid')
plt.show()

# 3. Kinematic models

<span style="font-weight: bold; color: #FF0000;">⚠ To be updated</span>

Kinematic models are now available through the WALLABY database. Only a subset of the sources have kinematic models. They can be accessed in a similar way to source products. First, we list the tags with `wallaby.get_kinematic_model_tags()`. Then, we use the tag to retrieve the catalog as an Astropy table with `wallaby.get_kinematic_model()`.

The kinematic model tags use the convention `TR` for "team release" and have the "Kin" keyword, whereas the source finding models use `DR` for "data release". To retrieve the kinematic models associated with NGC 5044 DR2, you would select the kinematic model tag "NGC 5044 Kin TR2".

In [ ]:
# Select the list of available team release tags

query = """SELECT DISTINCT team_release_kin
        FROM wallaby.kinematic_model"""
result = tap.search(query)
result

In [ ]:
# Set the kinematic tag desired
kin_tag = "NGC5044 Kin TR3"

# The generic query
query = """SELECT * FROM wallaby.kinematic_model k
        WHERE k.team_release_kin IN ('$TAG_NAME')"""
query = query.replace('$TAG_NAME', kin_tag)

# Run the tap query
result = tap.search(query)

# Get the resulting table
kin_table = result.to_table()

# Print off the table
kin_table

We can then use the kinematic model table in the same way as the source finding table. Below is an example figure created from the kinematic model catalog.

The profiles in the kinematic tables are stored as strings, so these will need to be parsed into arrays.  The function below is designed to do this.

In [ ]:
import numpy as np

def StrToArr(StrArr):
    StrVals = StrArr.split(",")
    FloatArr = np.zeros(len(StrVals))
    for i in range(len(StrVals)):
        FloatArr[i] = float(StrVals[i])
    return FloatArr

In [ ]:
%matplotlib inline

#To make the plot look a bit nicer, we'll set the figure size and font size
plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams["font.size"] = 16

for row in kin_table:
    r = StrToArr(row['rad'])
    v = StrToArr(row['vrot_model'])
    plt.plot(r, v, ls='-', color='k', marker='')
    
plt.title(kin_tag)
plt.xlabel(r"$R$ ('') ")
plt.ylabel(r"$V_{rot}$ (km/s)")
plt.xlim(0.0, 200)
plt.ylim(0, 350)
plt.grid(True)
plt.show()

The kinematic_model table points to the WKAPP models.  These only include the pilot fields.  For the 3KIDNAS models, we need to point to the kinematic_model_3kidnas table.  This will include all internally released kinematic models for the full survey

In [ ]:
# Select the list of available team release tags

query = """SELECT DISTINCT team_release_kin
        FROM wallaby.kinematic_model_3kidnas """
result = tap.search(query)
result

In [ ]:
kin_tag = "NGC 5044 E KinTR3"

# The generic query
query= """SELECT d.source_name, k.*
        FROM wallaby.kinematic_model_3kidnas k
        LEFT JOIN wallaby.detection d ON k.detection_id = d.id
        WHERE k.team_release_kin IN ('$TAG_NAME')"""
query = query.replace('$TAG_NAME', kin_tag)

# Run the tap query
result = tap.search(query)

# Get the resulting table
kin_table_3kidnas = result.to_table()

# Print off the table
kin_table_3kidnas

Once again we can make a plot of these models following the same approach as before. However, the strings include square brackets that need to be stripped before parsing.

In [ ]:
def StrToArr_3kidnas(StrArr):
    StrVals = StrArr.strip('[')
    StrVals = StrVals.strip(']')
    FloatArr = StrToArr(StrVals)
    return FloatArr

In [ ]:
%matplotlib inline

#To make the plot look a bit nicer, we'll set the figure size and font size
plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams["font.size"] = 16

for row in kin_table_3kidnas:
    r=StrToArr_3kidnas(row['rad'])
    v = StrToArr_3kidnas(row['vrot_model'])
    plt.plot(r, v, ls='-', color='k', marker='')
    
plt.title(kin_tag)
plt.xlabel(r"$R$ ('') ")
plt.ylabel(r"$V_{rot}$ (km/s)")
plt.xlim(0.0, 200)
plt.ylim(0, 350)
plt.grid(True)
plt.show()

The 3KIDNAS models also include estimates for R_HI and V_HI for a subset of galaxies.  These are indicated by when the rhi_flag==0.  These are calculated both in arcseconds and in kpc, and should already account for beam smearing.  However, it is very important to note that the kpc value is based on the dist_model value.  That distance is derived from a basic Hubble flow with H0=70 km/s/Mpc.  For detailed analysis, we strongly recommend rederiving the distances using some flow model (that wasn't derived with WALLABY values).  

With these caveats in mind, we can quickly plot the size-velocity scaling relation.

In [ ]:
%matplotlib inline

#To make the plot look a bit nicer, we'll set the figure size and font size
plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams["font.size"] = 16
fig=plt.figure()
ax = fig.add_subplot(1, 1, 1)

for row in kin_table_3kidnas:
    if row['rhi_flag']==0:
        DHI=2*row['rhi_kpc']
        DHI_Low=2*row['rhi_low_kpc']
        DHI_High=2*row['rhi_high_kpc']
        DHI_Err=[[DHI-DHI_Low],[DHI_High-DHI]]
        VHI=row['vhi']
        VHI_Err=row['e_vhi']
        
        ax.errorbar(VHI, DHI, xerr=VHI_Err, yerr=DHI_Err, ls='', color='k', marker='.')
    
#plt.title(kin_tag)
ax.set_xlabel(r"$V_{HI}$ (km/s) ")
ax.set_ylabel(r"$D_{HI}$ (kpc)")
ax.set_xscale('log')
ax.set_yscale('log')
plt.show()